In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

drive_content = os.listdir()
print(drive_content)

#Este path cambio
path = '/content/drive/MyDrive/Colab Notebooks'
json_path = '/content/drive/MyDrive/Colab Notebooks/cocotext.v2.json'

#print("ACA PATH:")
#print(path) #Se imprime el path para verificar que este sea el correcto
print(os.listdir(path=path)) #Se imprime el contenido del path para chequear que estamos parados en el lugar correcto

with open(json_path, 'r') as file:
    json_content = json.load(file)

# Explorar la estructura del archivo JSON
print(json_content.keys())

# Acceder a la lista de imágenes
images = json_content['imgs']
print("Número total de imágenes:", len(images))

# Acceder a la lista de anotaciones de texto
annotations = json_content['anns']
print("Número total de anotaciones de texto:", len(annotations))

max_sequence_length = 0

for annotation in annotations:
    if isinstance(annotation, str):  # Comprobar si la anotación es una cadena
        text_length = len(annotation)
        if text_length > max_sequence_length:
            max_sequence_length = text_length

print(max_sequence_length)

unique_chars = set()

# Itera sobre tus datos para recopilar todos los caracteres únicos
for annotation in annotations:
    if isinstance(annotation, str):  # Comprobar si la anotación es una cadena
        unique_chars.update(annotation)

# Calcula el tamaño del vocabulario como el número de caracteres únicos
vocab_size = len(unique_chars)

print(vocab_size)

Mounted at /content/drive
['.config', 'drive', 'sample_data']
['cocotext.v2.json', 'Reconocimiento de numeros manuscrito.ipynb', 'bacco-poc.ipynb', 'link datasets.gdoc', 'celsius.ipynb', 'nosolid-bacco.ipynb', 'Generador de dataset.ipynb']
dict_keys(['cats', 'anns', 'imgs', 'imgToAnns', 'info'])
Número total de imágenes: 53686
Número total de anotaciones de texto: 201126
6
10


In [84]:
#!pip install --upgrade pip
#!pip install tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

#model = models.Sequential([
#    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
#    layers.MaxPooling2D((2, 2)),
#    layers.Conv2D(64, (3, 3), activation='relu'),
#    layers.MaxPooling2D((2, 2)),
#    layers.Conv2D(64, (3, 3), activation='relu'),
#    layers.Flatten(),
#    layers.Dense(64, activation='relu'),
#    layers.Dense(10, activation='softmax')  # 10 clases para MNIST (0-9)
#])

#model = models.Sequential([
#    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(None, None, 3)),
#    layers.MaxPooling2D((2, 2)),
#    layers.Conv2D(64, (3, 3), activation='relu'),
#    layers.MaxPooling2D((2, 2)),
#    layers.Conv2D(64, (3, 3), activation='relu'),
#    layers.GlobalAveragePooling2D(),  # Capa de agrupación global
#    layers.Dense(64, activation='relu'),
#    layers.Dense(1, activation='sigmoid')
#])


model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.RepeatVector(max_sequence_length),  # Repetir vector para introducir la secuencia en la capa recurrente
    layers.LSTM(256, return_sequences=True),  # Capa LSTM para generar secuencias de caracteres
    layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax'))  # Capa de salida para generar la distribución de probabilidad de cada carácter
])


In [86]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', "loss"])

In [87]:
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
#import coco_text

  Cloning https://github.com/philferriere/cocoapi.git to /tmp/pip-req-build-javvoxkb
  Running command git clone --filter=blob:none --quiet https://github.com/philferriere/cocoapi.git /tmp/pip-req-build-javvoxkb
  Resolved https://github.com/philferriere/cocoapi.git to commit 2929bd2ef6b451054755dfd7ceb09278f935f7ad
  Preparing metadata (setup.py) ... done


In [88]:
!pip install fiftyone
import fiftyone as fo

# Create an empty dataset
dataset = fo.Dataset("test-dataset")

print(dataset)

import fiftyone as fo
import fiftyone.zoo as foz

#
# Only the required images will be downloaded (if necessary).
# By default, only detections are loaded
#

dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["validation","train"],
    classes=["person", "car"],
    # max_samples=50,
)

# Visualize the dataset in the FiftyOne App
session = fo.launch_app(dataset)

ValueError: Dataset name 'test-dataset' is not available

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
from fiftyone import ViewField as F

dataset = foz.load_zoo_dataset("cifar10", split="test")

cats = dataset.match(F("ground_truth.label") == "cat")
fob.compute_uniqueness(cats)

similar_cats = cats.sort_by("uniqueness", reverse=False)

session = fo.launch_app(view=similar_cats)

INFO:fiftyone.zoo.datasets:Downloading split 'test' to '/root/fiftyone/cifar10/test'


100%|██████████| 170498071/170498071 [00:02<00:00, 71761070.68it/s]


Extracting /root/fiftyone/cifar10/tmp-download/cifar-10-python.tar.gz to /root/fiftyone/cifar10/tmp-download
 100% |█████████████| 10000/10000 [5.9s elapsed, 0s remaining, 1.6K samples/s]        


INFO:eta.core.utils: 100% |█████████████| 10000/10000 [5.9s elapsed, 0s remaining, 1.6K samples/s]        


Dataset info written to '/root/fiftyone/cifar10/info.json'


...

INFO:fiftyone.brain.internal.core.uniqueness:Computing uniqueness...


Uniqueness computation complete


INFO:fiftyone.brain.internal.core.uniqueness:Uniqueness computation complete



Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.23.8

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



INFO:fiftyone.core.session.session:
Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.23.8

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|

